In [1]:
import numpy as np
import pandas as pd
from sympy.combinatorics.permutations import Permutation
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
def evolui_SEIR(pop, fator_de_transmissao, fator_de_recuperacao):
    indices = [i for i in range(len(pop))]
    p = np.random.permutation(indices)
    ciclos = Permutation(p).cyclic_form
    #print(ciclos)
    new_pop = pop.copy()
    not_in_ciclos = list(set(indices)-set([c for ciclo in ciclos for c in ciclo]))
    #print(not_in_ciclos)
    for ciclo in ciclos:
        for a in ciclo:
            n = ciclo.index(a)
            if pop[a] == 2 and np.random.rand() <= fator_de_recuperacao:
                new_pop[a] = 1
            else:
                if pop[a] == 2 and pop[ciclo[(n-1)%len(ciclo)]] == 1 and np.random.rand() <= fator_de_transmissao:
                    new_pop[ciclo[(n-1)%len(ciclo)]] = 2
                if pop[a] == 2 and pop[ciclo[(n+1)%len(ciclo)]] == 1 and np.random.rand() <= fator_de_transmissao:
                    new_pop[ciclo[(n+1)%len(ciclo)]] = 2
                    
    for x in not_in_ciclos:
        if pop[x] == 2 and np.random.rand() <= fator_de_recuperacao:
            new_pop[x] = 1
            continue
    #print(pop)
    return np.array(new_pop)

In [3]:
evolui_SEIR([1,1,1,1,1,1,1,2,2], 1, 0)

array([2, 2, 1, 2, 1, 2, 1, 2, 2])

In [4]:
def simulador(CI, fator_de_transmissao, fator_de_recuperacao, tmax, simuls):
    dados_frame = [go.Frame(data = (1+simuls)*[go.Scatter(x = np.array([0]), y = np.array(CI[1]), mode = 'lines'),
                                               go.Scatter(x = np.array([0]), y = np.array(CI[2]), mode = 'lines')])]
    
    populacoes = simuls*[CI[0]]
    num_suscetiveis  = [[CI[1]] for i in range(simuls)]
    num_infectados   = [[CI[2]] for i in range(simuls)]
    suscetiveis_mean = [CI[1]] + [0 for i in range(tmax-1)]
    infectados_mean  = [CI[2]] + [0 for i in range(tmax-1)]
    for t in range(1, tmax):
        dados = []
        for i in range(simuls):
            populacoes[i] = evolui_SEIR(populacoes[i], fator_de_transmissao, fator_de_recuperacao)
            num_suscetiveis[i] += [np.count_nonzero(populacoes[i] == 1)]
            num_infectados[i] += [np.count_nonzero(populacoes[i] == 2)]        
            #print(len(num_suscetiveis[i]))
            dados += [go.Scatter(x = np.arange(0,t+1), y = np.array(num_suscetiveis[i]), mode = 'lines'),
                      go.Scatter(x = np.arange(0,t+1), y = np.array(num_infectados[i]), mode = 'lines')]
        suscetiveis_mean[t] = 1/simuls * sum([num_suscetiveis[i][t] for i in range(simuls)])
        infectados_mean[t] = 1/simuls * sum([num_infectados[i][t] for i in range(simuls)])
        #print(i, num_infectados[i][t], infectados_mean[t], t)
        dados.extend([go.Scatter(x = np.arange(0,t+1), y = np.array(suscetiveis_mean[:t+1]), mode = 'lines'),
                      go.Scatter(x = np.arange(0,t+1), y = np.array(infectados_mean[:t+1]), mode = 'lines')])
        dados_frame += [go.Frame(data=dados)]
        #dadosi = go.Frame(data = [data_sus[t + k*tmax], data_inf[t + k*tmax], data_rec[t + k*tmax]])
        #dados.append(dadosi)
#dados = [go.Frame(data = [data_sus[i], data_inf[i], data_rec[i]]) for i in range(tmax*simuls)]
    return dados_frame

In [10]:
fator_de_transmissao = 0.15
fator_de_recuperacao = 0.15 # entre seis e sete dias de recuperação, em média

# estado inicial da população
num_individuos = 100
num_infectados0 = 10
populacao0 = np.ones(num_individuos)
infectados0 = np.random.choice(num_individuos, num_infectados0)
populacao0[infectados0] = 2*np.ones(num_infectados0)

# dias para cada evolução
tmax = 60
dias = np.arange(tmax)

# número de simulações
num_simulacoes = 20
CI = [populacao0, num_individuos-num_infectados0, num_infectados0]
# alpha para os gráficos
alpha = min(0.2, 5/num_simulacoes)

In [11]:
data_init = (num_simulacoes)*[go.Scatter(x=[0], y=np.array(CI[1]), mode = 'lines', name = 'Suscetiveis', legendgroup = 'Suscetiveis', marker = dict(color = 'rgba(0,0,252,'+str(alpha)+')'), showlegend= False),
                            go.Scatter(x=[0], y=np.array(CI[2]), mode = 'lines', name = 'Infectados', legendgroup = 'Infectados', marker = dict(color = 'rgba(252,0,0,'+str(alpha)+')'), showlegend= False)]

In [12]:
data_init += [go.Scatter(x=[0], y=np.array(CI[1]), mode = 'lines', name = 'Suscetiveis', legendgroup = 'Suscetiveis', marker = dict(color = 'blue')),
             go.Scatter(x=[0], y=np.array(CI[2]), mode = 'lines', name = 'Infectados', legendgroup = 'Infectados', marker = dict(color = 'red'))]

In [13]:
%%time
data = simulador(CI, fator_de_transmissao, fator_de_recuperacao,  tmax, num_simulacoes)

Wall time: 14 s


In [14]:
%%time
fig = go.Figure(
    data=data_init,
    layout=go.Layout(
        xaxis=dict(range=[-2, tmax + 2], autorange=False),
        yaxis=dict(range=[-2, 100 + 2], autorange=False),
        title=dict(text= f'SIS model', x = 0.5),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=data
)

fig.show()

Wall time: 6.96 s
